<a href="https://colab.research.google.com/github/ykitaguchi77/YOLACT_Mobius/blob/main/Inference_YOLOv8_instance_segmentation_kit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import cv2
from google.colab import drive
import shutil
import os

#driveのマウント
drive.mount('/content/drive')

!git clone https://github.com/ultralytics/ultralytics
%cd ultralytics

#yolov8
from ultralytics import YOLO
model = YOLO("/content/drive/MyDrive/Deep_learning/YOLACT_Mobius_ocular_dataset/sample_dataset_for_YOLOv8/last.pt")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
fatal: destination path 'ultralytics' already exists and is not an empty directory.
/content/ultralytics/ultralytics



image 1/1 /content/drive/MyDrive/Deep_learning/YOLACT_Mobius_ocular_dataset/sample_dataset_for_YOLOv8/images/val/1_1i_Lu_2.jpg: 384x640 1 eyelid, 1 iris, 1 pupil, 113.4ms
Speed: 14.7ms preprocess, 113.4ms inference, 1298.8ms postprocess per image at shape (1, 3, 384, 640)
Results saved to /content/ultralytics/runs/segment/predict


In [14]:
# ディレクトリパスを設定
image_dir = "/content/drive/MyDrive/Deep_learning/YOLACT_Mobius_ocular_dataset/sample_dataset_for_YOLOv8/images/val"

# ディレクトリ内の画像ファイルを取得
image_files = [os.path.join(image_dir, f) for f in os.listdir(image_dir) if f.endswith(('.jpg', '.jpeg', '.png'))]

img_path = image_files[14]
results = model(img_path, save=True)  # predict on an image


image 1/1 /content/drive/MyDrive/Deep_learning/YOLACT_Mobius_ocular_dataset/sample_dataset_for_YOLOv8/images/val/3_1n_Ll_1.jpg: 384x640 1 eyelid, 1 iris, 1 pupil, 21.0ms
Speed: 3.3ms preprocess, 21.0ms inference, 3.3ms postprocess per image at shape (1, 3, 384, 640)
Results saved to /content/ultralytics/runs/segment/predict2


In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# 画像を読み込む
img = cv2.imread(img_path)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# 結果を取得
result = results[0]

# マスクとボックスを取得
masks = result.masks.data.cpu().numpy()
boxes = result.boxes.data.cpu().numpy()

# 各クラスに対して異なる色を設定
colors = [(255, 0, 0), (0, 255, 0), (0, 0, 255)]  # 赤、緑、青

# セグメンテーションマスクを重ねる
for i, mask in enumerate(masks):
    color = colors[int(boxes[i, 5]) % len(colors)]
    mask = mask.astype(np.uint8) * 255

    # マスクを元の画像のサイズにリサイズ
    mask = cv2.resize(mask, (img.shape[1], img.shape[0]))

    colored_mask = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    colored_mask[mask > 0] = color

    # マスクを画像にブレンド (50%の透過度)
    img = cv2.addWeighted(img, 1, colored_mask, 0.5, 0)

# バウンディングボックスを描画
for box in boxes:
    x1, y1, x2, y2 = map(int, box[:4])
    cls = int(box[5])
    cv2.rectangle(img, (x1, y1), (x2, y2), colors[cls % len(colors)], 2)

# 結果を表示
plt.figure(figsize=(12, 8))
plt.imshow(img)
plt.axis('off')
plt.title('YOLOv8 Segmentation Result')
plt.show()